<a href="https://colab.research.google.com/github/bjimenezTechnodomus/licitaciones/blob/main/clasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gspread

import pandas as pd
import re

gc = gspread.service_account()

In [23]:
worksheet = gc.open('Total partidas').sheet1

rows = worksheet.get_all_values()

data = pd.DataFrame.from_records(rows)
data.columns = data.iloc[0]
data.drop(0, inplace=True)
display(data.info())
display(data.describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118779 entries, 1 to 118779
Data columns (total 19 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   Id_Partida                118779 non-null  object
 1   Archivo_Origen            118779 non-null  object
 2   Consecutivo               118779 non-null  object
 3   Número del Procedimiento  118779 non-null  object
 4   Tipo de Procedimiento     118779 non-null  object
 5   Convocante                118779 non-null  object
 6   Par- tida                 118779 non-null  object
 7   Cantidad Mínima           118779 non-null  object
 8   Cantidad Máxima           118779 non-null  object
 9   Unidad                    118779 non-null  object
 10  Descripción               118779 non-null  object
 11  Descripción de la Clave   118779 non-null  object
 12  Observaciones             118779 non-null  object
 13  Grupo de productos        118779 non-null  object
 14  Esta

None

,Id_Partida,Archivo_Origen,Consecutivo,Número del Procedimiento,Tipo de Procedimiento,Convocante,Par- tida,Cantidad Mínima,Cantidad Máxima,Unidad,Descripción,Descripción de la Clave,Observaciones,Grupo de productos,Estaba equivocada,Cuadro Básico,Nos interesa,Razones,Grupo Productos
count,118779,118779,118779,118779,118779,118779,118779,118779,118779,118779,118779,118779,118779,118779,118779,118779,118779,118779,118779
unique,117224,58,5428,4009,5,445,1184,1275,5271,3725,82576,13,4,14,4,15040,2,50,36
top,11cNMT7STJJ9BS-1dCK-Rxx6ilFLFmvHB8kqhFfPnj8Y00...,1OAk_gdJojr40Hi2f8FNzChB_Psr7ZQYQ82Nw7ozUUpI,1,LA-008IZC999-E632-2021,Adjudicación Directa,IMSS-Coordinación de Abastecimiento y Equipami...,1,,1,,"060.168.6637.11 SONDAS DE LATEX, PUNTA REDONDA...",Material de Curación,,No incluye,No,,,,
freq,2,5428,60,1089,46615,5382,3606,100398,20122,18977,59,67198,118105,116522,118150,49261,99464,90345,116985


In [25]:
data = data[['Id_Partida', 'Convocante', 'Descripción', 'Descripción de la Clave', 'Grupo de productos', 'Cuadro Básico', 'Nos interesa', 'Grupo Productos']]
data.columns = ['id', 'convocante', 'descripcion', 'clave', 'grupo_1', 'cuadro_basico', 'interes', 'grupo_2']
data['cuadro_2'] = data['descripcion'].str.extract(r'(\d{3}[\.\s-]\d{3}[\.\s-]\d{4})').replace(r'[\s-]', '.')
CATEGORIAS = ['grupo_1', 'clave', 'interes', 'grupo_2']
data['grupo_2'] = data['grupo_2'].str.lower()
data['descripcion'] = data['descripcion'].str.lower()
data['grupo_2'][data['grupo_2'] == 'desinfectantes'] = 'lavado'
data['grupo_2'][data['grupo_2'] == 'contenedores'] = 'lavado'
data['grupo_2'][data['grupo_2'] == 'lavado y cuidado de instrumental'] = 'lavado'
data['grupo_2'][data['grupo_2'] == 'lavadoras termodesinfectoras'] = 'lavado'
data['grupo_2'][data['grupo_2'] == 'película y químicos dental'] = 'filme plano'
data['grupo_2'][data['grupo_2'] == 'lavadoras ultrasónicas'] = 'lavado'
data['grupo_2'][data['grupo_2'] == 'lavadoras ultrasonicas'] = 'lavado'
data['grupo_2'][data['grupo_2'] == 'reveladoras y casetas análogas'] = 'filme plano'
data['grupo_2'][data['grupo_2'] == 'bolsa / rollo material no tejido'] = 'envoltura material no tejido'
data['grupo_2'][data['grupo_2'] == 'cr'] = 'rayos x digital'
data['grupo_2'][data['grupo_2'] == 'mesa de rayos x'] = 'rayos x digital'
data['grupo_2'][data['grupo_2'] == 'mastografias'] = 'mastografía'
data['grupo_2'][data['grupo_2'] == 'pelicula dry'] = 'película dry'
data.loc[49804, 'grupo_2'] == 'indicadores biológicos'
data.loc[72386, 'grupo_2'] == 'indicadores biológicos'
data['grupo_2'][data['grupo_2'] == 'retos ind. quimico'] = 'indicadores biológicos'
data['grupo_2'][data['grupo_2'] == 'bolsa / rollo material no tejido'] = 'envoltura material no tejido'
data[CATEGORIAS] = data[CATEGORIAS].apply(lambda x: x.astype('category'), axis=0)
categorias_dumies = pd.get_dummies(data[CATEGORIAS])
grupo_tech = pd.get_dummies(data['grupo_1'])

KeyError: "None of [Index(['Id_Partida', 'Convocante', 'Descripción', 'Descripción de la Clave',\n       'Grupo de productos', 'Cuadro Básico', 'Nos interesa',\n       'Grupo Productos'],\n      dtype='object')] are in the [columns]"

In [26]:
data['grupo_2'].value_counts()

                                116985
bolsa / rollo mixta papel          772
indicadores biológicos             198
vapor                              118
indicadores químicos               114
plasma                             112
mastografía                         89
indicadores biologicos              66
lavado                              62
rayos x digital                     42
selladoras                          38
envoltura material no tejido        35
bolsa / rollo tyvek                 32
indicadores quimicos                30
filme plano                         26
película dry                        24
incubadoras                         20
licenciamiento                      16
Name: grupo_2, dtype: int64

In [27]:
y = data['grupo_2']
X = data.drop('grupo_2', axis=1)
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(X, y):
  X_train = X.iloc[train_index]
  X_test = X.iloc[test_index]
  y_train = y.iloc[train_index]
  y_test = y.iloc[test_index]


In [37]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

tokens = '[A-Za-z0-9]+(?=\\s+)'

X_train['descripcion'].fillna('', inplace=True)

vectorizer = CountVectorizer(token_pattern= tokens)
vectorizer.fit_transform(X_train['descripcion'])

pl = Pipeline([
    ('vec', CountVectorizer(token_pattern = tokens)),
    ('clf', OneVsRestClassifier(LogisticRegression(max_iter=200)))
])

pl.fit(X_train['descripcion'], y_train)

Pipeline(steps=[('vec', CountVectorizer(token_pattern='[A-Za-z0-9]+(?=\\s+)')),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(max_iter=200)))])

In [70]:
import numpy as np

data2 = data.sort_values('grupo_2')
no_producto = data2.iloc[0:116984]
productos = data2.iloc[116985:]
drops = np.random.choice(no_producto.index, 100000, replace=False)
no_producto.drop(drops, inplace=True)
data2 = pd.concat([no_producto, productos])
X2 = data2.drop('grupo_2', axis=1)
y2 = data2['grupo_2']
X2['descripcion'].fillna('', inplace=True)
vec = CountVectorizer(token_pattern=tokens)
vec.fit_transform(X_train['descripcion'])
pl.fit(X2['descripcion'], y2)


/home/mauricio/.local/lib/python3.9/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Pipeline(steps=[('vec', CountVectorizer(token_pattern='[A-Za-z0-9]+(?=\\s+)')),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(max_iter=200)))])

In [74]:
data2[['descripcion', 'cuadro_2', 'grupo_2']].to_csv('datos_entrenamiento.csv', index=False)


# Clasificación con Spacy

Una prueba para mejorar el modelo usando los modelos preentrenados de spacy

In [ ]:
# Ejemplo usando Spacy para la clasificacción de las partidas
!pip install --upgrade spacy

In [8]:
import spacy
nlp = spacy.blank('es')
textcat = nlp.add_pipe('textcat')
labels = data['grupo_2'].unique()
for label in labels:
  textcat.add_label(label)

train_text = data['descripcion'].values
train_labels = [{'cats': {label: case == label for label in labels}} for case in data['grupo_2']]
train_data = list(zip(train_text, train_labels))
train_data[:3]

In [10]:
from spacy.util import minibatch
from spacy.training.example import Example 

spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

batches = minibatch(train_data, size=10)
for batch in batches:
  for text, labels in batch:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, labels)
    nlp.update([example], sgd=optimizer)

## Ejemplo de aplicación del modelado

Este es un ejemplo del uso del algoritmo en un archivo anterior de partidas.

In [13]:
ejemplo = gc.open_by_key('1MJEn7aqCjBaMq2EQ2x1VhKgRZkgKeXpKQw0VFQ8P9HQ')
ejemplo = ejemplo.get_worksheet(1)
rows = ejemplo.get_all_values()
ejemplo = pd.DataFrame.from_records(rows)
ejemplo.columns = ejemplo.iloc[0]
ejemplo.drop(0, inplace=True)
ejemplo['Descripción'] = ejemplo['Descripción'].str.lower()
display(ejemplo.describe())

,Número del Procedimiento,Tipo de Procedimiento,Convocante,Par- tida,Cantidad Mínima,Cantidad Máxima,Unidad,Descripción,Descripción de la Clave,Observaciones
count,770,770,770,770,770,770,770,770,770,770
unique,33,5,21,365,1,145,182,768,9,2
top,LP-919044992-I35-2017,Licitación,SERVICIOS DE SALUD DE NUEVO LEON 0.P.D.,1,,2,FRASCO C/1,080.830.5247.10.01 violeta cristal (violeta de...,Material de Laboratorio,
freq,365,371,365,27,770,133,123,2,510,768


In [16]:
textos = ejemplo['Descripción']
docs = [nlp.tokenizer(texto) for texto in textos]
textcat = nlp.get_pipe('textcat')
scores = textcat.predict(docs)
predicted_labels = scores.argmax(axis=1)
predicted_labels = [textcat.labels[label] for label in predicted_labels]
ejemplo['prediccion'] = predicted_labels

# Los resultados de las predicciones fueron peores que con regresión lógistica de sklearn con un tiempo de entrenamiento de 75 min.

array([[9.9969506e-01, 1.4746693e-07, 1.6233599e-06, ..., 1.6796745e-05,
        2.3630530e-05, 4.1592764e-07],
       [9.9986792e-01, 8.1916262e-08, 1.0703052e-06, ..., 2.5192519e-06,
        6.2908748e-06, 1.4878000e-07],
       [9.9967253e-01, 2.0727890e-07, 2.6657037e-06, ..., 2.8320401e-05,
        2.6798980e-05, 5.0911331e-07],
       ...,
       [9.9989688e-01, 7.0644006e-08, 6.1960850e-07, ..., 4.8927755e-06,
        4.7161811e-06, 1.1914614e-07],
       [9.9981803e-01, 1.8959325e-07, 1.2679784e-06, ..., 1.5117265e-05,
        4.0799237e-06, 3.5056394e-07],
       [9.9989343e-01, 1.4030205e-07, 1.1366303e-06, ..., 8.4441390e-06,
        2.7949463e-06, 2.4647318e-07]], dtype=float32)

In [71]:
ejemplo['prediccion2'] = pl.predict(ejemplo['Descripción'])
ejemplo

,Número del Procedimiento,Tipo de Procedimiento,Convocante,Par- tida,Cantidad Mínima,Cantidad Máxima,Unidad,Descripción,Descripción de la Clave,Observaciones,prediccion,prediccion2
1,LP-919044992-I35-2017,Licitación,SERVICIOS DE SALUD DE NUEVO LEON 0.P.D.,1,,24,EQUIPO C/96,800812141 (partida 1 sustancias quimicas ren.1...,Material de Laboratorio,,,
2,LP-919044992-I35-2017,Licitación,SERVICIOS DE SALUD DE NUEVO LEON 0.P.D.,2,,7,PLACA C/1,806101481 (partida 1 sustancias quimicas ren.2...,Material de Laboratorio,,,
3,LP-919044992-I35-2017,Licitación,SERVICIOS DE SALUD DE NUEVO LEON 0.P.D.,3,,7,EQUIPO C/96,800702508 (partida 1 sustancias quimicas ren.3...,Material de Laboratorio,,,
4,LP-919044992-I35-2017,Licitación,SERVICIOS DE SALUD DE NUEVO LEON 0.P.D.,4,,4,TUBOS C/10,5019994934 (partida 1 sustancias quimicas ren....,Material de Laboratorio,,,
5,LP-919044992-I35-2017,Licitación,SERVICIOS DE SALUD DE NUEVO LEON 0.P.D.,5,,2,FRASCO C/1,800810590 (partida 1 sustancias quimicas ren.5...,Material de Laboratorio,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
766,AA-019GYR079-E160-2017,Adjudicación Directa,"IMSS-Unidad Médica de Alta Especialidad, Hospi...",4,,12,25 CJA RLL,060.203.0397.11.01 cintas para uso quirurgico ...,Material de Curación,,,
767,AA-019GYR079-E161-2017,Adjudicación Directa,"IMSS-Unidad Médica de Alta Especialidad, Hospi...",1,,20,16 ENV LTO,060.066.0773.12.01 antisepticos. alcohol desna...,Material de Curación,,,
768,AA-019GYR079-E161-2017,Adjudicación Directa,"IMSS-Unidad Médica de Alta Especialidad, Hospi...",2,,1,33491 PZA PZA,"060.681.0059.11.01 pavales de forma anatomica,...",Material de Curación,,,
769,AA-03890O001-E211-2017,Adjudicación Directa,CIBNOR-Subdirección de Recursos Materiales #03...,1,,1,,(mat. laboratorio ren.1) (sticky stuff) 1 shee...,Material de Laboratorio,,,


In [75]:
todas = pd.read_csv('partidas_solicitante2.csv')
todas['categoria'] = pl.predict(todas['descripcion'])

In [76]:
todas.to_csv('partidas_etiquetadas.csv', index=False)

# Procesado de los datos de Cantidades

In [ ]:
textos = [text for text in data['Cantidad Máxima'] if len(text) > 5]

rango_maxmin = r'(\d+) a (\d+)'
cantidad_unidad = r'(\d+) (\w+)'
minmax = r'min+max'
dos_palabras = r'([A-Z]+) +([A-Z]+)'
miles = r'(\d{1,3}),(\d{3})'

display(textos)

In [ ]:
display([re.match(cantidad_unidad, texto).group(2) for texto in textos if re.match(cantidad_unidad, texto)])

In [ ]:
# Funciones utilitarias

def num_text_to_int(text):
  nums = ['cero', 'uno', 'dos', 'tres', 'cuatro', 'cinco', 'seis', 'siete', 'ocho', 'nueve', 'diez']

In [ ]:
data['Cantidad Máxima'] = (data['Cantidad Máxima'].str.replace('\D', ''))
data['Cantidad Mínima'] = (data['Cantidad Mínima'].str.replace('\D', ''))
data['Cantidad Máxima'] = data['Cantidad Máxima'].astype(int)
data['Cantidad Mínima'] = data['Cantidad Mínima'].astype(int)

In [ ]:
for index, num in enumerate(data['Cantidad Máxima']):
  print('{} {}'.format(index, int(num)))